## Toronto Event Planning

### Introduction:


**Rockstar Event Planners** is a company based in Toronto for organizing events and booking the hotels and resturants for the visitors.It mostly works with the companies in Toronto that bring in the employees working in its offshore branches for the global events. The company must arrange the place of accomodation. The halls of meetings must be arranged for the employees in the morning. In the evenings the employees and their families are taken out to visit the landscapes play some games and ends with a dinner at resturants.


### Data Description
The data used in this project is provided by Foursquare location data as required. The data is grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.




### Import the Libraries

First all the libraries needed for this project is imported here.


In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from pandas.io.json import json_normalize

import folium # for plotting the maps


Since we are using the foursquare api for getting the location data, we will set up the foursquare api.


In [2]:
ClIENT_ID = '*********' # your Foursquare ID
ClIENT_SECRET = '********' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30


### Get the Coordinates of the city of Toronto

Here we will obtain the coordinates of the city of Toronto. The coordinates will then be used in the foursqare API

In [3]:
city = 'Toronto'
geolocator = Nominatim(user_agent="toronto_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print("Latitude:",latitude," Longitude:",longitude)

Latitude: 43.6534817  Longitude: -79.3839347


Now using the search query we will make the calls to the foursquare API to get the **Hotels** upto around of 500 units

In [4]:
#get the hotels

query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, query, radius, LIMIT)


In [5]:
results = requests.get(url).json()
venues = results['response']['venues']


dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1588960595,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51d212c3498ebf27dc469bc9,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,Chelsea Hotel,v-1588960595,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ae61cf6f964a520caa421e3,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,Pantages Hotel & Spa,v-1588960595,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51e48697498eded9073c6c17,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,Crew Room Eaton Chelsea Hotel,v-1588960595,NaN
4,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,4b68aed1f964a520de862be3,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,The Rex Hotel Jazz & Blues Bar,v-1588960595,62225795


Now lets do one of the most important step in the data science process, that is the **data cleaning**. Here we will extract the data to get into meaningful values. At last the unnecessary columns are dropped.

In [6]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)

clean_dataframe.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
3,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON


### Remove the NaN values. 

Now we will remove the NaN values. These values will cause issues while visualizing the data. So remove the using the dropna method

In [7]:
clean_dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
clean_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
12,Pantages Hotel Gym,Gym,200 Victoria St.,43.654464,-79.379036,M5B 1V8,ON
15,The Spa At The Adelaide Hotel,Spa,325 Bay St,43.649725,-79.380346,M5H 4G5,ON
16,Pantages Hotel Toronto Centre,Residential Building (Apartment / Condo),200 Victoria Street,43.655205,-79.379182,M5B 1V8,ON
20,"Chelsea Hotel, Toronto Meetings & Events Venue",Convention Center,33 Gerrard Street West,43.658446,-79.382985,M5G 1Z4,ON




In the above dataframe we can see in the categories column that there are many categories. Some of these are not needed for our analysis. For example the residential area is not needed for the employees and their families who visit the place for short time. So we will keep the following categories.

* Hotel
* Jazz Club
* Convention Center
* Event Space

In [8]:
array= ['Hotel', 'Convention Center', 'Jazz Club', 'Event Space']
hotel_dataframe= clean_dataframe.loc[clean_dataframe['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
20,"Chelsea Hotel, Toronto Meetings & Events Venue",Convention Center,33 Gerrard Street West,43.658446,-79.382985,M5G 1Z4,ON
21,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
22,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
26,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON


Now it is important to remove the hotel names where there is a duplicate hotel name.

In [9]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
20,"Chelsea Hotel, Toronto Meetings & Events Venue",Convention Center,33 Gerrard Street West,43.658446,-79.382985,M5G 1Z4,ON
21,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
22,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
26,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON




Now, the most interesting part. The Rockstar company is interested to allocate the hotels to the employees such that the hotel assigned is there in the same Postal Code as that of Convention Center or Event Space. Lets select the event space and the postal code is **M5H 2M9**. Luckily there is a hotel in the same postal code. Lets group them together

In [10]:
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
29,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON




Now as an event planner in the Rockstar company, the planner have to plan an outing for the employees and their families. What is more better than a park? So lets get the park data for Toronto. For this we will get the data and follow the same process we followed for hotels

In [11]:
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [12]:
parks = requests.get(url).json()

venues = parks['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

clean_park_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
6,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
7,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
8,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
10,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
14,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
15,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
19,Regent Park Community Centre,Community Center,465 Dundas St E,43.659512,-79.363048,M5A 2B2,ON
20,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON


We will now remove the venues other than the parks. This will only keep the relevant data.

In [13]:
df_park = clean_park_dataframe[clean_park_dataframe.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
8,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
14,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
15,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
20,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
21,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
27,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON


As per the requirement we will search for and obtain the dataframe similar to above for the following data
* Shopping Centers
* Restaurants
* Cafetaria


#### Shopping Centers

In [14]:
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [15]:
sresults = requests.get(url).json()
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

#remove the unwanted rows
clean_Shopping_dataframe= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)

clean_Shopping_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


We will keep the shopping malls and remove the department store data

In [16]:
df_Shopping = clean_Shopping_dataframe[clean_Shopping_dataframe.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


#### Cafetarias

In [17]:
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [18]:
cresults = requests.get(url).json()

venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]
clean_Cafeteria_dataframe= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)

clean_Cafeteria_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
5,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
6,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,NaN,ON
7,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
8,SickKids Cafeteria,Food Court,555 University Avenue,43.657584,-79.375652,NaN,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON


There are so many other categories. So lets remove those and only keep cafeterias

In [19]:
df_Cafeteria = clean_Cafeteria_dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
16,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
17,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
24,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
27,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON


#### Resturants

In [20]:
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [24]:
Rresults = requests.get(url).json()

# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
df_Restaurant = clean_Restaurant_dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
4,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
6,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
7,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
8,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
9,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
10,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
12,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON


## Clustering of these dataframes together to visualize the clusters and make the decisions.

#### Plan1

In this plan the employees and family are taken to the cafeteria for some snaks. Then they move to the park and then finally go to the restaurant to finish the day. Lets group the cafeteria, restaurant and park and visualize the cluster.

In [25]:
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)

In [29]:
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)
color = ['blue','red','green']
i=0
for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color[1],
        fill=True,
        fill_color=color[1],
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map



#### Plan2

In this plan the employee and their families can be taken to the cafeteria. Then they can go to the shopping stores. Then finally they go to the resturants for the dinner.

In [30]:
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map




## Conclusion:

Looking at the above maps we can see that the cluster for plan 2 is a bit farther away. So this plan while good will lead to more time spent in travelling. So if I were to choose, i will select the plan 1.

So we can take the visitors to cafeteria. Then we go to the park for recreational activities. Then we finally go to the hotels for the dinner.